In [1]:
# """FINALIZE DATSET"""


# # organize dataset into a useful structure
# from os import makedirs
# from os import listdir
# from shutil import copyfile
# # create directories
# dataset_home = 'finalize_dogs_vs_cats/'
# # create label subdirectories
# labeldirs = ['dogs/', 'cats/']
# for labldir in labeldirs:
#     newdir = dataset_home + labldir
#     makedirs(newdir, exist_ok=True)
# # copy training dataset images into subdirectories
# src_directory = 'dogs-vs-cats/train/'
# for file in listdir(src_directory):
#     src = src_directory + '/' + file
#     if file.startswith('cat'):
#         dst = dataset_home + 'cats/'  + file
#         copyfile(src, dst)
#     elif file.startswith('dog'):
#         dst = dataset_home + 'dogs/'  + file
#         copyfile(src, dst)

In [2]:
import sys
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

Using TensorFlow backend.


In [3]:
# define cnn model
def define_model():
    # load model
    model = VGG16(include_top=False, input_shape=(224, 224, 3))
    # mark loaded layers as not trainable
    for layer in model.layers:
        layer.trainable = False    
    # add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)    
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(1, activation='sigmoid')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model
 

In [4]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')  
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    pyplot.savefig(filename + '_plot.png')
    pyplot.close()

In [5]:
"""TEST HARNESS FOR EVALUATION"""


# run the test harness for evaluating a model
def run_test_harness():
    # define model
    model = define_model()
    # create data generator
    datagen = ImageDataGenerator(featurewise_center=True)
    # specify imagenet mean values for centering    
    datagen.mean = [123.68, 116.779, 103.939]
    # prepare iterator
    train_it = datagen.flow_from_directory('dataset_dogs_vs_cats/train/',
        class_mode='binary', batch_size=64, target_size=(224, 224))
    test_it = datagen.flow_from_directory('dataset_dogs_vs_cats/test/',
        class_mode='binary', batch_size=64, target_size=(224, 224))
    # fit model
    history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
        validation_data=test_it, validation_steps=len(test_it), epochs=10, verbose=1)
    # evaluate model
    _, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=1)
    print('> %.3f' % (acc * 100.0))
    # learning curves
    summarize_diagnostics(history)

In [6]:
# """TEST HARNESS FOR SAVING A MODEL"""


# def run_test_harness():
# # define model
# 	model = define_model()
# 	# create data generator
# 	datagen = ImageDataGenerator(featurewise_center=True)
# 	# specify imagenet mean values for centering
# 	datagen.mean = [123.68, 116.779, 103.939]
# 	# prepare iterator
# 	train_it = datagen.flow_from_directory('finalize_dogs_vs_cats/',
# 		class_mode='binary', batch_size=64, target_size=(224, 224))
# 	# fit model
# 	model.fit_generator(train_it, steps_per_epoch=len(train_it), epochs=10, verbose=1)
# 	# save model
# 	model.save('final_model.h5')

In [7]:
# entry point, run the test harness
run_test_harness()


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Found 18697 images belonging to 2 classes.
Found 6303 images belonging to 2 classes.

Epoch 1/10
  5/293 [..............................] - ETA: 40:59 - loss: 7.0719 - accuracy: 0.7531

KeyboardInterrupt: 